# 🔗 Prosess-Topologi og Avhengighetsanalyse

Denne notebooken viser hvordan NeqSim kan:
1. **Trekke ut funksjonell rekkefølge** fra en prosessmodell
2. **Identifisere parallelt utstyr** (redundans)
3. **Merke utstyr med STID-tagger** (funksjonell lokasjon)
4. **Analysere avhengigheter** - "hvis denne pumpen går ut, hva må vi passe på?"
5. **Kryss-installasjon analyse** - Gullfaks → Åsgard

In [ ]:
# Initialiser NeqSim
import jpype
import jpype.imports

if not jpype.isJVMStarted():
    import neqsim

# Import Java klasser
from neqsim.thermo.system import SystemSrkEos
from neqsim.process.processmodel import ProcessSystem
from neqsim.process.equipment.stream import Stream
from neqsim.process.equipment.separator import Separator
from neqsim.process.equipment.compressor import Compressor
from neqsim.process.equipment.heatexchanger import Cooler
from neqsim.process.equipment.pump import Pump
from neqsim.process.util.topology import ProcessTopologyAnalyzer, FunctionalLocation, DependencyAnalyzer

import json
import pandas as pd

print("✅ NeqSim Topologi-analyse lastet!")

## 1. Bygg Prosessanlegg med Parallelt Utstyr

Vi lager et anlegg med to parallelle kompressortog (A og B).

In [ ]:
# Lag fluid
fluid = SystemSrkEos(280.0, 50.0)
fluid.addComponent("methane", 0.85)
fluid.addComponent("ethane", 0.10)
fluid.addComponent("propane", 0.05)
fluid.setMixingRule("classic")

# Bygg prosess
process = ProcessSystem()

# Innløpsstrøm
feed = Stream("Well Stream", fluid)
feed.setFlowRate(20000.0, "kg/hr")
feed.setTemperature(40.0, "C")
feed.setPressure(40.0, "bara")
process.add(feed)

# HP Separator
separator = Separator("HP Separator", feed)
process.add(separator)

# Kompressor Tog A
compressorA = Compressor("Compressor Train A", separator.getGasOutStream())
compressorA.setOutletPressure(100.0, "bara")
process.add(compressorA)

# Kompressor Tog B (parallelt - bruker kopi av strømmen)
# I praksis ville dette vært en splitter - her forenkler vi
compressorB = Compressor("Compressor Train B", separator.getGasOutStream())
compressorB.setOutletPressure(100.0, "bara")
process.add(compressorB)

# Kjøler etter kompressor A
coolerA = Cooler("Aftercooler A", compressorA.getOutletStream())
coolerA.setOutTemperature(30.0, "C")
process.add(coolerA)

# Kjøler etter kompressor B
coolerB = Cooler("Aftercooler B", compressorB.getOutletStream())
coolerB.setOutTemperature(30.0, "C")
process.add(coolerB)

# Eksportstrøm
export = Stream("Export Gas", coolerA.getOutletStream())
process.add(export)

# Kondensatpumpe
pump = Pump("Condensate Pump", separator.getLiquidOutStream())
pump.setOutletPressure(60.0, "bara")
process.add(pump)

process.run()
print(f"✅ Prosess kjørt - Eksport: {export.getFlowRate('kg/hr'):.0f} kg/hr")

## 2. Bygg Topologi (Graf-struktur)

In [ ]:
# Lag topologi-analysator
topology = ProcessTopologyAnalyzer(process)

# Bygg topologi-grafen
topology.buildTopology()

# Vis noder
print("📊 PROSESS-TOPOLOGI")
print("="*60)
print(f"Antall enheter: {len(topology.getNodes())}")
print(f"Antall koblinger: {len(topology.getEdges())}")
print()

# Vis topologisk rekkefølge
print("🔢 FUNKSJONELL REKKEFØLGE:")
order = topology.getTopologicalOrder()
sorted_units = sorted(order.items(), key=lambda x: x[1])
for name, pos in sorted_units:
    node = topology.getNode(name)
    print(f"  {pos}. {name} ({node.getEquipmentType()})")

## 3. Legg til STID Funksjonelle Lokasjoner

STID-format: `PPPP-TT-NNNNN[S]`
- PPPP = Installasjons-kode (1775 = Gullfaks C)
- TT = Utstyrs-type (KA=kompressor, PA=pumpe, VG=separator)
- NNNNN = Sekvensnummer
- S = Tog-suffix (A, B, C...)

In [ ]:
# Definer STID-tagger for utstyr
topology.setFunctionalLocation("HP Separator", "1775-VG-23001")
topology.setFunctionalLocation("Compressor Train A", "1775-KA-23011A")
topology.setFunctionalLocation("Compressor Train B", "1775-KA-23011B")
topology.setFunctionalLocation("Aftercooler A", "1775-WC-23021A")
topology.setFunctionalLocation("Aftercooler B", "1775-WC-23021B")
topology.setFunctionalLocation("Condensate Pump", "1775-PA-24001")

# Vis utstyr med STID
print("🏷️ FUNKSJONELLE LOKASJONER (STID):")
print("="*70)
for name, node in topology.getNodes().items():
    loc = node.getFunctionalLocation()
    if loc:
        print(f"  {loc.getFullTag():20} | {name:25} | {loc.getInstallationName()}")
    else:
        print(f"  {'(ingen STID)':20} | {name:25} |")

## 4. Identifiser Parallelt Utstyr

NeqSim kan automatisk finne parallelt utstyr basert på:
1. Samme type utstyr
2. Samme inngang/utgang
3. STID-tagger med samme base (A, B suffix)

In [ ]:
# Vis parallelle grupper
print("🔀 PARALLELT UTSTYR (Redundans):")
print("="*60)

parallel_groups = list(topology.getParallelGroups())
if parallel_groups:
    for i, group in enumerate(parallel_groups, 1):
        group_list = list(group)
        print(f"\n  Gruppe {i}:")
        for eq in group_list:
            node = topology.getNode(eq)
            stid = node.getFunctionalLocation().getFullTag() if node.getFunctionalLocation() else "N/A"
            print(f"    - {eq} ({stid})")
else:
    print("  (Ingen automatisk oppdaget - sjekk STID-tagger)")

# Sjekk STID-basert parallellitet
print("\n🔗 STID-BASERT PARALLELLITET:")
compA_loc = FunctionalLocation("1775-KA-23011A")
compB_loc = FunctionalLocation("1775-KA-23011B")
print(f"  {compA_loc} og {compB_loc} er parallelle: {compA_loc.isParallelTo(compB_loc)}")

## 5. Avhengighetsanalyse - "Hvis X går ut, hva må vi passe på?"

Dette er kjernen i spørsmålet ditt!

In [ ]:
# Lag avhengighets-analysator
deps = DependencyAnalyzer(process, topology)

# Analyser hva som skjer hvis Kompressor A går ned
print("⚠️ SCENARO: Kompressor Tog A går ned")
print("="*70)

result = deps.analyzeFailure("Compressor Train A")

print(f"\n🔴 Feilet utstyr: {result.getFailedEquipment()}")
if result.getFailedLocation():
    print(f"   STID: {result.getFailedLocation().getFullTag()}")
    print(f"   Installasjon: {result.getFailedLocation().getInstallationName()}")

print("\n📍 DIREKTE PÅVIRKET (stopper umiddelbart):")
for eq in result.getDirectlyAffected():
    print(f"   - {eq}")
if not result.getDirectlyAffected():
    print("   (ingen direkte nedstrøms)")

print("\n📍 INDIREKTE PÅVIRKET (kaskade-effekt):")
for eq in result.getIndirectlyAffected():
    print(f"   - {eq}")
if not result.getIndirectlyAffected():
    print("   (ingen indirekte)")

print(f"\n💰 Estimert produksjonstap: {result.getTotalProductionLoss():.1f}%")

In [ ]:
# Hva må vi passe ekstra på?
print("🔍 UTSTYR SOM MÅ OVERVÅKES EKSTRA:")
print("="*70)

to_monitor = deps.getEquipmentToMonitor("Compressor Train A")
for eq, reason in to_monitor.items():
    print(f"  {eq}")
    print(f"    └─ {reason}")

print("\n⚡ ØKET KRITIKALITET:")
for eq, crit in result.getIncreasedCriticality().items():
    print(f"  {eq}: {crit:.2f} (0-1 skala)")

## 6. Kryss-Installasjon Avhengigheter

Definerer hvordan utstyr på én plattform påvirker andre plattformer.

Eksempel: Gullfaks C eksporterer gass som prosesseres på Åsgard.

In [ ]:
# Legg til kryss-installasjon avhengighet
# Gullfaks C eksportkompressor → Åsgard A innløpsseparator
deps.addCrossInstallationDependency(
    "Export Gas",           # Kilde på Gullfaks C
    "Åsgard Inlet Separator", # Mål på Åsgard A
    "Åsgard A",             # Installasjon
    "gas_export"            # Type avhengighet
)

# Også med STID-tagger
source_stid = FunctionalLocation("1775-KA-23011A")  # Gullfaks C kompressor
target_stid = FunctionalLocation("2540-VG-30001")   # Åsgard A separator
deps.addCrossInstallationDependency(source_stid, target_stid, "gas_export", 0.6)

print("🌐 KRYSS-INSTALLASJON AVHENGIGHETER DEFINERT:")
print(f"  {source_stid} ({source_stid.getInstallationName()})")
print(f"    └─► {target_stid} ({target_stid.getInstallationName()})")
print(f"        Type: gas_export, Impact: 60%")

In [ ]:
# Analyser kryss-installasjon effekter
print("\n⚠️ KRYSS-INSTALLASJON EFFEKTER ved feil på Export Gas:")
print("="*70)

cross_result = deps.analyzeFailure("Export Gas")
for effect in cross_result.getCrossInstallationEffects():
    print(f"  ➡️ Påvirker: {effect.getTargetEquipment()}")
    print(f"     Installasjon: {effect.getTargetInstallation()}")
    print(f"     Type: {effect.getDependencyType()}")
    print(f"     Impact: {effect.getImpactFactor()*100:.0f}%")
    print()

## 7. Eksporter som Graf (DOT format)

Kan visualiseres med Graphviz.

In [ ]:
# Eksporter som DOT-graf
dot_graph = topology.toDotGraph()
print("📊 GRAPHVIZ DOT FORMAT:")
print("="*60)
print(dot_graph)

In [ ]:
# Eksporter som JSON
json_data = json.loads(topology.toJson())
print("📋 JSON EKSPORT:")
print(json.dumps(json_data, indent=2))

## 8. Oppsummering

NeqSim kan nå:

| Funksjon | Beskrivelse |
|----------|-------------|
| **Topologi** | Trekker ut graf-struktur fra prosessmodell |
| **STID-tagging** | Merker utstyr med funksjonell lokasjon |
| **Parallelt utstyr** | Identifiserer redundans automatisk |
| **Avhengigheter** | "Hvis X feiler, hva må vi passe på?" |
| **Kryss-installasjon** | Gullfaks ↔ Åsgard avhengigheter |
| **Kritikalitet** | Beregner hvilke enheter som blir mer kritiske |

### Neste steg:
- Integrer med SAP/STID-database for automatisk tagging
- Koble mot K-Spice modeller
- Bygge full felt-topologi (Gullfaks + Åsgard + ...)